# Estimating Queue Priority Wait Times

https://dl.acm.org/doi/10.1145/3427921.3450249

In [40]:
import pandas
from scipy.optimize import newton

In [41]:
P_GROUP_BOUNDS = [1,3,10]

COLS_TO_USE = [4,5,6,9,10,12,13,14,15,16]
HEADER_NAMES = ['size','weight','received_time','fee','confirmed_block_height','confirm_time','waiting_time','fee_rate','enter_block_height','no_block_confirm']

### Data Loading

In [42]:
d1 = pandas.read_csv('TimetxinBlock621500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d2 = pandas.read_csv('TimetxinBlock622000.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d3 = pandas.read_csv('TimetxinBlock622500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
dataFrame = pandas.concat([d1, d2, d3])

dataFrame = dataFrame[dataFrame['waiting_time'] >= 0] # Remove any rows with negative values for waiting_time
print("Transaction Count: ", len(dataFrame))
dataFrame.head(10)

Transaction Count:  3389521


,size,weight,received_time,fee,confirmed_block_height,confirm_time,waiting_time,fee_rate,enter_block_height,no_block_confirm
0,223,892,1583793568,2486,621001,1583794214,646,11.147982,621000,2
1,223,892,1583791819,2486,621001,1583794214,2395,11.147982,620998,4
2,223,892,1583793885,2486,621001,1583794214,329,11.147982,621001,1
3,223,892,1583793700,2486,621001,1583794214,514,11.147982,621000,2
4,223,892,1583791354,2486,621001,1583794214,2860,11.147982,620998,4
5,223,892,1583791731,2486,621001,1583794214,2483,11.147982,620998,4
6,371,1484,1583792024,4136,621001,1583794214,2190,11.148248,620998,4
7,371,1484,1583793603,4136,621001,1583794214,611,11.148248,621000,2
8,371,1484,1583793883,4136,621001,1583794214,331,11.148248,621001,1
9,371,1484,1583791921,4136,621001,1583794214,2293,11.148248,620998,4


### Block Size

In [43]:
block_groups = dataFrame.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))
print("Mean Block Size: ", mean_block_size)

Mean Block Size:  2266.0


### Fee Rate Priority Range

In [44]:
def calculate_feerate_for_priority_groups(data, upper_boundary):
    feerate_range = []
    transaction_counts = 0
    feerate_range.append(data['fee_rate'].max())
    for i in range(len(P_GROUP_BOUNDS)):
        transaction_counts = len(data[data['no_block_confirm'] <= upper_boundary[i]])
        #boundary = data['fee_rate'].nlargest(transaction_counts).iloc[-1]
        boundary = data['fee_rate'].tail(transaction_counts).iloc[0]
        feerate_range.append(boundary)
    return feerate_range

p_groups_feerate_range = calculate_feerate_for_priority_groups(dataFrame.sort_values('fee_rate'), P_GROUP_BOUNDS)
p_groups_feerate_range.append(0.0)
print("one block waited ration: ", p_groups_feerate_range)

one block waited ration:  [52380.95238095238, 30.175438596491237, 18.529411764705888, 5.0, 0.0]


### Arrival Rates (Lambda) - Priority

In [45]:
p_groups_lambda = []
transaction_count = len(dataFrame)
received_time_max = dataFrame['received_time'].max()
received_time_min = dataFrame['received_time'].min()

for current, next in zip(p_groups_feerate_range,p_groups_feerate_range[1:]):
    p_df = dataFrame[(dataFrame['fee_rate'] <= current) & (dataFrame['fee_rate'] > next)]
    p_transaction_count = len(p_df)
    #p_lambda = float(p_transaction_count) / (float(p_df['received_time'].max()) - float(p_df['received_time'].min()))
    p_lambda = float(p_transaction_count) / (received_time_max - received_time_min)
    p_groups_lambda.append(p_lambda)
    
    print("Priority Group " + str(p_groups_feerate_range.index(current)+1))
    print("Transaction Count: ", p_transaction_count)
    print("Feerate Min: ", float(p_df['fee_rate'].min()))
    print("Feerate Max: ", float(p_df['fee_rate'].max()))
    print("Lambda: " + str(p_lambda) + "\n")
    
print("Sum Of All Lambda: ", sum(p_groups_lambda))

Priority Group 1
Transaction Count:  2035967
Feerate Min:  30.175580221997983
Feerate Max:  52380.95238095238
Lambda: 1.851985231178986

Priority Group 2
Transaction Count:  565691
Feerate Min:  18.52950075642965
Feerate Max:  30.175438596491237
Lambda: 0.5145718852078014

Priority Group 3
Transaction Count:  385863
Feerate Min:  5.000028548752705
Feerate Max:  18.529411764705888
Lambda: 0.3509941847085032

Priority Group 4
Transaction Count:  402000
Feerate Min:  0.907563025210084
Feerate Max:  5.0
Lambda: 0.36567295193583804

Sum Of All Lambda:  3.083224253031129


### Service Rate (Mu)

In [46]:
service_time = 600
mu = 1/service_time # Our Original Mu = 0.0016666666666666668
#mu = (1/service_time)*mean_block_size # Our New Mu = 3.776666666666667
#mu = mean_block_size / service_time # = 3.776666666666667
print("Mu: ", mu)

Mu:  0.0016666666666666668


### Validation Of Z Using Newton Method (Mine Vs SciPy)

In [47]:
my_p_group_z = []
sciPy_p_group_z = []
lambda_sum = 0.0

def NewtonMethod(lam, m_u, block_size, x0, epsilon):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    while True:
        x1 = x0 - fx(x0) / dfx(x0)
        t = abs(x1 - x0)
        if t < epsilon:
            break
        x0 = x1
    return x0

def SciPyNewton(lam, m_u, block_size, x0, epsilon, max_iteration):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    #dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    #return newton(fx, x0, fprime=dfx, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)
    return newton(fx, x0, fprime=None, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)

for i in range(len(p_groups_lambda)):
    lambda_sum += p_groups_lambda[i]
    my_p_group_z.append(NewtonMethod(lambda_sum, mu, mean_block_size, 0, 1e-10))
    sciPy_p_group_z.append(SciPyNewton(lambda_sum, mu, mean_block_size, 0, 1e-10, 500))

for i in range(len(my_p_group_z)):
    print("Priority Group " + str(i+1))
    print("My Z: \n" + str(my_p_group_z[i]))
    print("SciPy Z: \n" + str(sciPy_p_group_z[i]) + "\n")
    

Priority Group 1
My Z: 
0.9992742750913941
SciPy Z: 
0.9992742751058932

Priority Group 2
My Z: 
0.9995497987137135
SciPy Z: 
0.9995497987138171

Priority Group 3
My Z: 
0.9996918411807911
SciPy Z: 
0.9996918412628326

Priority Group 4
My Z: 
0.9998145619120352
SciPy Z: 
0.9998145619140734



### Wait Times Using Z

In [48]:
prev_transaction_in_queue = 0.0
p_group_response_time = []

for i in range(len(p_groups_lambda)):
    temp = sciPy_p_group_z[i] / (1 - sciPy_p_group_z[i]) # L(y) = z(x) / (1 - z(x))
    transaction_in_queue = temp - prev_transaction_in_queue # L(x) = L(y) - L(y-1)
    prev_transaction_in_queue = temp
    
    #transaction_in_queue = my_p_group_z[i] / (1 - my_p_group_z[i]) - prev_transaction_in_queue # L(x) = (z(x) / (1 - z(x))) - L(x-1)
    #prev_transaction_in_queue = transaction_in_queue

    response_time = transaction_in_queue / p_groups_lambda[i] # W(x) = L(x) / Lambda(x)

    p_group_response_time.append(response_time)

    print("Priority Group " + str(i+1))
    print("Z: ", sciPy_p_group_z[i])
    print("Average Transaction In Queue (L): ", transaction_in_queue)
    print("Average Response Time In Queue (W): " + str(response_time) + "\n")

Priority Group 1
Z:  0.9992742751058932
Average Transaction In Queue (L):  1376.9326134743274
Average Response Time In Queue (W): 743.4900615259027

Priority Group 2
Z:  0.9995497987138171
Average Transaction In Queue (L):  843.2960473290768
Average Response Time In Queue (W): 1638.8303977947135

Priority Group 3
Z:  0.9996918412628326
Average Transaction In Queue (L):  1023.8521348672389
Average Response Time In Queue (W): 2917.0059775136638

Priority Group 4
Z:  0.9998145619140734
Average Transaction In Queue (L):  2147.554676179624
Average Response Time In Queue (W): 5872.883583023225

